In [1]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

def calculate_ear(eye):
    """Calculate the eye aspect ratio (EAR) for a given eye."""
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def draw_eye_landmarks(frame, eye_landmarks):
    """Draw eye landmarks on the frame."""
    for point in eye_landmarks:
        cv2.circle(frame, point, 2, (0, 255, 0), -1)

def main():
    # Threshold for eye aspect ratio to indicate blink
    EYE_AR_THRESH = 0.3
    # Number of consecutive frames the eye must be below the threshold
    EYE_AR_CONSEC_FRAMES = 48

    # Initialize the frame counters and the total number of blinks
    COUNTER = 0

    # Load the pre-trained models for face and facial landmark detection
    detector = dlib.get_frontal_face_detector()
    try:
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    except RuntimeError:
        print("Error: 'shape_predictor_68_face_landmarks.dat' file not found.")
        print("Download the file from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
        return

    # Start video capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image from camera. Exiting...")
            break

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = detector(gray)

        for face in faces:
            x1, y1 = face.left(), face.top()
            x2, y2 = face.right(), face.bottom()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (102, 0, 153), 3)

            # Determine the facial landmarks for the face region
            landmarks = predictor(gray, face)
            left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
            right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])

            draw_eye_landmarks(frame, left_eye)
            draw_eye_landmarks(frame, right_eye)

            leftEAR = calculate_ear(left_eye)
            rightEAR = calculate_ear(right_eye)
            ear = (leftEAR + rightEAR) / 2.0

            # Check if the eye aspect ratio is below the blink threshold
            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    cv2.putText(frame, "YORGUNLUK ALGILANDI!", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                COUNTER = 0

        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
